In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import nltk
from nltk.corpus import brown
from nltk.corpus import stopwords
import re

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [3]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4060 Laptop GPU'

In [4]:
# Загрузка корпуса Brown
nltk.download('brown')

# Получение предложений из корпуса Brown
sentences = brown.sents()
print(sentences[:5])  # Вывод первых 5 предложений для примера


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ['The', 'September-October', 'term', 'jury', 'had', 'been', 'charged', 'by', 'Fulton', 'Superior', 'Court', 'Judge', 'Durwood', 'Pye', 'to', 'investigate', 'reports', 'of', 'possible', '``', 'irregularities', "''", 'in', 'the', 'hard-fought', 'primary', 'which', 'was', 'won', 'by', 'Mayor-nominate', 'Ivan', 'Allen', 'Jr.', '.'], ['``', 'Only', 'a', 'relative', 'handful', 'of', 'such', 'rep

[nltk_data] Downloading package brown to /home/kolya/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kolya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import re

def preprocess_text(text):
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Удаление пунктуаци
    words = [word for word in text.split() if word not in stop_words]
    return words

# Применение предобработки
preprocessed_sentences = [preprocess_text(' '.join(sentence)) for sentence in sentences]
print(preprocessed_sentences[:5])  # Вывод первых 5 предложений для примера

[['fulton', 'county', 'grand', 'jury', 'said', 'friday', 'investigation', 'atlantas', 'recent', 'primary', 'election', 'produced', 'evidence', 'irregularities', 'took', 'place'], ['jury', 'said', 'termend', 'presentments', 'city', 'executive', 'committee', 'overall', 'charge', 'election', 'deserves', 'praise', 'thanks', 'city', 'atlanta', 'manner', 'election', 'conducted'], ['septemberoctober', 'term', 'jury', 'charged', 'fulton', 'superior', 'court', 'judge', 'durwood', 'pye', 'investigate', 'reports', 'possible', 'irregularities', 'hardfought', 'primary', 'mayornominate', 'ivan', 'allen', 'jr'], ['relative', 'handful', 'reports', 'received', 'jury', 'said', 'considering', 'widespread', 'interest', 'election', 'number', 'voters', 'size', 'city'], ['jury', 'said', 'find', 'many', 'georgias', 'registration', 'election', 'laws', 'outmoded', 'inadequate', 'often', 'ambiguous']]


In [6]:
all_words = [word for sentence in preprocessed_sentences for word in sentence]
dictionary = sorted(list(set(all_words)))
unique_words_array = np.array(dictionary).reshape(-1, 1)
print(len(dictionary))

46138


In [7]:
unique_words_array

array([['aa'],
       ['aaa'],
       ['aaaee'],
       ...,
       ['zurich'],
       ['zwei'],
       ['zworykin']], dtype='<U28')

In [8]:
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder()
onehot.fit(unique_words_array)

OneHotEncoder()

In [9]:
onehot.transform(unique_words_array).shape

(46138, 46138)

In [10]:
def vectorize_word(word, encoder):
    if word in dictionary:
        word_array = np.array([[word]])
        return torch.tensor(encoder.transform(word_array).toarray())
    else:
        return None

In [11]:
vectorize_word('woman', onehot).shape

torch.Size([1, 46138])

In [12]:
#мы научились векторизовывать слова

In [13]:
#Начинаем работу с Pytorch

In [14]:
preprocessed_sentences[0]

['fulton',
 'county',
 'grand',
 'jury',
 'said',
 'friday',
 'investigation',
 'atlantas',
 'recent',
 'primary',
 'election',
 'produced',
 'evidence',
 'irregularities',
 'took',
 'place']

In [23]:
m = nn.Softmax(dim = 0)
m(torch.randn(5, 1))


tensor([[0.0714],
        [0.6375],
        [0.1013],
        [0.1181],
        [0.0717]])

In [25]:
class MyWord2Vec(nn.Module):
    def __init__(self, embedding_layers = 128, dict_len = 46138):
        super(MyWord2Vec, self).__init__()
        self.l1 = nn.Linear(dict_len, embedding_layers)
        self.l2 = nn.Linear(embedding_layers, dict_len)
        #self.softmax = nn.Softmax(dim = 1)
        self.double()
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        #x = self.softmax(x)
        return x
    def embedding(self, word, vectorizer):
        return self.l1(vectorizer(word))

In [26]:
model = MyWord2Vec(embedding_layers = 32).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [33]:
num_epochs = 10

from tqdm import tqdm

for epoch in range(num_epochs):
    
    model.train()
    
    for count, batch in tqdm(enumerate(preprocessed_sentences[:5])):

        
        
        for i in range(2, len(batch)-2):
            
            for j in range(1,3):
                #контекстное слово слева
                word = batch[i]
                context_word = batch[i-j]
                word_tensor = vectorize_word(word, onehot).to(device)
                context_tensor = vectorize_word(context_word, onehot).to(device)

                outputs = model(word_tensor)
                loss = criterion(outputs, context_tensor)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()


                #контекстное слово справа
                context_word = batch[i+j]
                context_tensor = vectorize_word(context_word, onehot).to(device)
                outputs = model(word_tensor)
                loss = criterion(outputs, context_tensor)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        




    
                

5it [00:20,  4.08s/it]
5it [00:19,  3.91s/it]
5it [00:20,  4.02s/it]
5it [00:20,  4.11s/it]
5it [00:20,  4.00s/it]
5it [00:18,  3.75s/it]
5it [00:19,  3.82s/it]
5it [00:17,  3.54s/it]
5it [00:18,  3.62s/it]
5it [00:17,  3.56s/it]


In [43]:
vectorize_word('king', onehot).to(device)

tensor([[0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [48]:
model.l1(vectorize_word('king', onehot).to(device))

torch.Size([1, 32])

In [29]:
def w(word):
    return model.l1(vectorize_word(word, onehot).to(device))

In [50]:
w('king') - w('woman')

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0',
       dtype=torch.float64, grad_fn=<SubBackward0>)

In [30]:
w('man')

tensor([[-0.0794, -0.1409,  0.1548,  0.0259, -0.1623, -0.0458, -0.1407,  0.0023,
          0.0383, -0.0150, -0.1833,  0.0192, -0.0153, -0.0636, -0.0359, -0.0058,
          0.0034,  0.0354,  0.1760,  0.1846,  0.0384, -0.0941, -0.3143,  0.0487,
         -0.2195, -0.0947, -0.2119,  0.0397, -0.1401,  0.1065, -0.1754,  0.0037]],
       device='cuda:0', dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [32]:
w('friday')

tensor([[-0.0802, -0.1521,  0.1611,  0.0202, -0.1685, -0.0477, -0.1381, -0.0022,
          0.0380, -0.0061, -0.1845,  0.0190, -0.0155, -0.0676, -0.0351,  0.0006,
          0.0057,  0.0360,  0.1816,  0.1827,  0.0288, -0.0934, -0.3131,  0.0445,
         -0.2226, -0.0952, -0.2165,  0.0369, -0.1495,  0.1091, -0.1808,  0.0049]],
       device='cuda:0', dtype=torch.float64, grad_fn=<AddmmBackward0>)